In [1]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import argparse

import model
import evaluation

PATH_TO_TRAIN = '/Users/qiangwang/Data/recsys/yoochoose-data/yoochoose-clicks.dat'
PATH_TO_TEST = '/Users/qiangwang/Data/recsys/yoochoose-data/yoochoose-test.dat'

class Args():
    is_training = False
    layers = 1
    rnn_size = 100
    n_epochs = 3
    batch_size = 50
    dropout_p_hidden=1
    learning_rate = 0.001
    decay = 0.96
    decay_steps = 1e4
    sigma = 0
    init_as_normal = False
    reset_after_session = True
    session_key = 'SessionId'
    item_key = 'ItemId'
    time_key = 'Timestamp'
    grad_cap = 0
    test_model = 2
    checkpoint_dir = './checkpoint'
    loss = 'cross-entropy'
    final_act = 'softmax'
    hidden_act = 'tanh'
    n_items = -1

def parseArgs():
    parser = argparse.ArgumentParser(description='GRU4Rec args')
    parser.add_argument('--layer', default=1, type=int)
    parser.add_argument('--size', default=100, type=int)
    parser.add_argument('--epoch', default=3, type=int)
    parser.add_argument('--lr', default=0.001, type=float)
    parser.add_argument('--train', default=1, type=int)
    parser.add_argument('--test', default=2, type=int)
    parser.add_argument('--hidden_act', default='tanh', type=str)
    parser.add_argument('--final_act', default='softmax', type=str)
    parser.add_argument('--loss', default='cross-entropy', type=str)
    parser.add_argument('--dropout', default='0.5', type=float)
    
    return parser.parse_args()

In [2]:
data = pd.read_csv(PATH_TO_TRAIN, header=None, names=['SessionId', 'Timestamp', 'ItemId', 'Label'], sep=',', dtype={'ItemId': np.int64}, parse_dates=['Timestamp'])

/Users/qiangwang/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
data.describe()

,SessionId,ItemId
count,3.300394e+07,3.300394e+07
mean,5.620195e+06,2.168345e+08
std,3.356590e+06,2.981944e+07
min,1.000000e+00,2.145072e+08
25%,2.691097e+06,2.146761e+08
50%,5.516873e+06,2.148268e+08
75%,8.527383e+06,2.148454e+08
max,1.156216e+07,1.178838e+09


In [3]:
data.dtypes

SessionId             int64
Timestamp    datetime64[ns]
ItemId                int64
Label                object
dtype: object

In [4]:
args = Args()
args.n_items = len(data['ItemId'].unique())
args.layers = 1
args.rnn_size = 100
args.n_epochs = 3
args.learning_rate = 0.001
args.is_training = 1
args.test_model = 2
args.hidden_act = 'tanh'
args.final_act = 'softmax'
args.loss = 'cross-entropy'
args.dropout_p_hidden = 1.0 if args.is_training == 0 else 0.5
print(args.dropout_p_hidden)
if not os.path.exists(args.checkpoint_dir):
    os.mkdir(args.checkpoint_dir)
gpu_config = tf.ConfigProto()
gpu_config.gpu_options.allow_growth = True
with tf.Session(config=gpu_config) as sess:
    gru = model.GRU4Rec(sess, args)
    if args.is_training:
        gru.fit(data)
    else:
        res = evaluation.evaluate_sessions_batch(gru, data, valid)
        print('Recall@20: {}\tMRR@20: {}'.format(res[0], res[1]))

0.5
fitting model...
Epoch 0	Step 1	lr: 0.001000	loss: 3.912021
Epoch 0	Step 10000	lr: 0.001000	loss: 2.326884
Epoch 0	Step 20000	lr: 0.000960	loss: 2.040384
Epoch 0	Step 30000	lr: 0.000922	loss: 1.916124
Epoch 0	Step 40000	lr: 0.000885	loss: 1.822196
Epoch 0	Step 50000	lr: 0.000849	loss: 1.762607
Epoch 0	Step 60000	lr: 0.000815	loss: 1.710961
Epoch 0	Step 70000	lr: 0.000783	loss: 1.697033
Epoch 0	Step 80000	lr: 0.000751	loss: 1.681928
Epoch 0	Step 90000	lr: 0.000721	loss: 1.671708
Epoch 0	Step 100000	lr: 0.000693	loss: 1.658332
Epoch 0	Step 110000	lr: 0.000665	loss: 1.657692
Epoch 0	Step 120000	lr: 0.000638	loss: 1.653269
Epoch 0	Step 130000	lr: 0.000613	loss: 1.650745
Epoch 0	Step 140000	lr: 0.000588	loss: 1.645277
Epoch 0	Step 150000	lr: 0.000565	loss: 1.636330


KeyboardInterrupt: 